In [1]:
######################################################################################
# Using Electrodermal Activity to Detect Deception and Suspicion during a Card Game
# Affective Computing - Mini-project
# Jan Ondras
# Dec 2017 - Jan 2018
######################################################################################
#######################################################################
# Generate feature files with filenames of format: 
# features_{feature_type}_u_{epoch_delay}_{max_epoch_len}.csv
# u means unit normalization
#######################################################################

import glob
import csv
from scipy.signal import butter, filtfilt
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# File
def extract_features(feature_type, epoch_delay, max_deception_epoch_length, mean_susp_length, file_ID):

    fs = 32. # Hz - sampling frequency
    fc = 3.  # Hz - cutoff frequency

    norm_type = 1

    if norm_type == 0:
        norm_to_baseline = True
        znormalize = False
        unitnormalize = False
    elif norm_type == 1:
        norm_to_baseline = False
        znormalize = False
        unitnormalize = True
    else:
        norm_to_baseline = False
        znormalize = True
        unitnormalize = False

    apply_filter = True #False

    # If maximum suspicion epoch length is not provided, use (subject-dependent) estimates from stats of suspicion epoch lengths
    # this is not used => go to else
    if mean_susp_length == None:
        mean_susp_len = {'11': 1.5483870967741935, '10': 1.5, '13': 1.4047619047619047, '12': 1.6428571428571428, '15': 1.6666666666666667, '07': 1.8023255813953489, '17': 1.5714285714285714, '16': 1.1666666666666667, '19': 1.0, '18': 2.0, '08': 1.2592592592592593, '09': 2.25, '04': 2.0, '20': 2.0, '02': 3.0, '01': 3.0, '06': 2.7000000000000002, '05': 1.7142857142857142, '03': 2.2083333333333335, '14': 1.3333333333333333}
    else: # same for all
        mean_susp_len = {}
        for i in range(1, 21):
            mean_susp_len['{:02d}'.format(i)] = mean_susp_length
        
    # Normalize and filter whole EDA recording
    # def normalize_and_filter(y, baseline):
    #     #y = (y - y.mean()) / y.std() # z-score
    #     y = (y - baseline) / (max(y) - baseline) # [0,1]
    #     # For digital filters, Wn is normalized from 0 to 1, where 1 is the Nyquist frequency, pi radians/sample
    #     b, a = butter(N=32, Wn=2*fc/fs, btype='low', analog=False, output='ba')
    #     z = filtfilt(b, a, y)
    #     return z

    # Normalize EDA recording to the baseline of corresponding subject
    # def normalize_to_baseline(y, baseline):
    #     return (y - baseline) / (max(y) - baseline) 

    # Filter whole EDA recording
    def filt(y):
        # For digital filters, Wn is normalized from 0 to 1, where 1 is the Nyquist frequency, pi radians/sample
        b, a = butter(N=5, Wn=2*fc/fs, btype='low', analog=False, output='ba')
        z = filtfilt(b, a, y)
        return z

    # Z-norm EDA signal for a given epoch only
    def znorm(y):
        return (y - np.mean(y)) / np.std(y) # z-score

    # Normalize EDA signal to [0,1] for a given epoch only
    def unitnorm(y):
        return (y - np.min(y)) / (np.max(y) - np.min(y))

    # First return value tells whether to keep the sample
    def extract_features(data):
    #     print data
        if len(data) < 2:
            return False, []

        features = []

        # 1.) mean
        features.append(np.mean(data))

        # 2.) std
        std = np.std(data)
        features.append(std)
        if std == 0.:
            return False, []

        mafd = np.mean(np.abs(np.diff(data))) # mean absolute first differences, of the raw signal
        masd = np.mean(np.abs(np.subtract(data[2:], data[:-2]))) # mean absolute second differences, of the raw signal
        if np.isnan(mafd) or np.isnan(masd):
            return False, []
        else:
            mafdN = mafd / std  # mean absolute first differences, of the normalized signal
            masdN = masd / std  # mean absolute second differences, of the normalized signal  

        features.extend([mafd, mafdN, masd, masdN])

        return True, features

    # Delay epoch by a fixed amount at both endpoints
    def delay_epoch(epoch_start, epoch_end, delay, SID):
        # Input and output times are strings 
        # [:-3] removes microseconds
        dt_epoch_start = datetime.strptime(epoch_start, '%H:%M:%S.%f') # datetime object
        dt_epoch_end = datetime.strptime(epoch_end, '%H:%M:%S.%f') 
        # Shorten epochs longer than max_deception_epoch_length (comparing datetime objects)
        if dt_epoch_start < dt_epoch_end - timedelta(seconds=max_deception_epoch_length):
            dt_epoch_start = dt_epoch_end - timedelta(seconds=max_deception_epoch_length)
        # Delay both endpoints
        delayed_epoch_start = datetime.strftime(dt_epoch_start + timedelta(seconds=delay), '%H:%M:%S.%f')[:-3]
        delayed_epoch_end = datetime.strftime(dt_epoch_end + timedelta(seconds=delay), '%H:%M:%S.%f')[:-3]
        return delayed_epoch_start, delayed_epoch_end

    # Delay and threshold according to subject-specific mean suspicion epoch length
    def delay_susp_epoch(epoch_start, epoch_end, delay, SID):
        dt_epoch_start = datetime.strptime(epoch_start, '%H:%M:%S.%f') # datetime object
        dt_epoch_end = datetime.strptime(epoch_end, '%H:%M:%S.%f') 
        # Shorten epochs longer than subject-specific mean suspicion epoch length (mean_susp_len)
        if dt_epoch_start < dt_epoch_end - timedelta(seconds=mean_susp_len[SID]):
            dt_epoch_start = dt_epoch_end - timedelta(seconds=mean_susp_len[SID])
        # Delay both endpoints
        delayed_epoch_start = datetime.strftime(dt_epoch_start + timedelta(seconds=delay), '%H:%M:%S.%f')[:-3]
        delayed_epoch_end = datetime.strftime(dt_epoch_end + timedelta(seconds=delay), '%H:%M:%S.%f')[:-3]
        return delayed_epoch_start, delayed_epoch_end

    # Delay and threshold according to subject-specific mean suspicion epoch length
    def delay_trust_epoch(epoch_start, epoch_end, delay, SID):
        dt_epoch_start = datetime.strptime(epoch_start, '%H:%M:%S.%f') # datetime object
        dt_epoch_end = datetime.strptime(epoch_end, '%H:%M:%S.%f') 
        # Time interval when suspicion/trust happens is from epoch_start 
        # and it has length mean_susp_len[SID] if epoch_start + 2*mean_susp_len[SID] < epoch_end
        # otherwise (epoch_end - epoch_start) / 2
#         if dt_epoch_start + timedelta(seconds=mean_susp_len[SID]) < dt_epoch_end:
        if dt_epoch_start + timedelta(seconds=2.*mean_susp_len[SID]) < dt_epoch_end:
            dt_epoch_end = dt_epoch_start + timedelta(seconds=mean_susp_len[SID])
        else:
            dt_epoch_end = dt_epoch_start + timedelta(seconds=0.5*(dt_epoch_end-dt_epoch_start).total_seconds())        
        # Delay both endpoints
        delayed_epoch_start = datetime.strftime(dt_epoch_start + timedelta(seconds=delay), '%H:%M:%S.%f')[:-3]
        delayed_epoch_end = datetime.strftime(dt_epoch_end + timedelta(seconds=delay), '%H:%M:%S.%f')[:-3]
        return delayed_epoch_start, delayed_epoch_end

    deception_classes = ['deception', 'truth'] # 0 = deception, 1 = truth
    suspicion_classes = ['suspicion', 'trust'] # 0 = suspicion, 1 = trust

    features_deception = []
    features_suspicion = []

    numS = 0

    # Load EDA baseline epochs for each subject 
    # Structure of EDA baseline epochs file: 
    # Subject ID | Baseline epoch start (system) time | Baseline epoch end (system) time 
    # baselines = np.loadtxt('./../Experiment/EDA/BaselineEDA.csv', delimiter=",", skiprows=1, dtype=str)
    # baseline_epochs = dict(zip(baselines[:, 0], baselines[:, 1:]))
    # print "Baselines: \n", baseline_epochs

    # Iterate over games (10 in total)
    game_events_files = glob.glob('./../Experiment/AnnotatedEvents/G*.csv')
    for game_events_file_name in game_events_files:

        # Extract subject IDs of players
        P1_SID = game_events_file_name[-8:-6]
        P2_SID = game_events_file_name[-6:-4]
#         print "Extracting subjects: ", P1_SID, P2_SID

        # Load EDA files (fields are strings)
        P1_L_data = np.loadtxt('./../Experiment/EDA/S' + P1_SID + '_L.csv', delimiter=",", skiprows=8, dtype=str)
        P1_R_data = np.loadtxt('./../Experiment/EDA/S' + P1_SID + '_R.csv', delimiter=",", skiprows=8, dtype=str)
        P2_L_data = np.loadtxt('./../Experiment/EDA/S' + P2_SID + '_L.csv', delimiter=",", skiprows=8, dtype=str)
        P2_R_data = np.loadtxt('./../Experiment/EDA/S' + P2_SID + '_R.csv', delimiter=",", skiprows=8, dtype=str)

        # EDA values are no longer strings, but times are
        P1_L_times = P1_L_data[:,0]
        P1_L_EDA = P1_L_data[:,6].astype(np.float32)

        P1_R_times = P1_R_data[:,0]
        P1_R_EDA = P1_R_data[:,6].astype(np.float32)

        P2_L_times = P2_L_data[:,0]
        P2_L_EDA = P2_L_data[:,6].astype(np.float32)

        P2_R_times = P2_R_data[:,0]
        P2_R_EDA = P2_R_data[:,6].astype(np.float32)

        # Calculate EDA baselines - means
    #     P1_L_baseline = np.mean([EDA for t,EDA in zip(P1_L_times,P1_L_EDA) 
    #                         if (baseline_epochs[P1_SID][0] <= t) and (t <= baseline_epochs[P1_SID][1]) ])
    #     P1_R_baseline = np.mean([EDA for t,EDA in zip(P1_R_times,P1_R_EDA) 
    #                         if (baseline_epochs[P1_SID][0] <= t) and (t <= baseline_epochs[P1_SID][1]) ])
    #     P2_L_baseline = np.mean([EDA for t,EDA in zip(P2_L_times,P2_L_EDA) 
    #                         if (baseline_epochs[P2_SID][0] <= t) and (t <= baseline_epochs[P2_SID][1]) ])
    #     P2_R_baseline = np.mean([EDA for t,EDA in zip(P2_R_times,P2_R_EDA) 
    #                         if (baseline_epochs[P2_SID][0] <= t) and (t <= baseline_epochs[P2_SID][1]) ])

        # Filter
        if apply_filter:
            f_P1_L_EDA = filt(P1_L_EDA)
            f_P1_R_EDA = filt(P1_R_EDA)
            f_P2_L_EDA = filt(P2_L_EDA)
            f_P2_R_EDA = filt(P2_R_EDA)
        else:
            f_P1_L_EDA = P1_L_EDA
            f_P1_R_EDA = P1_R_EDA
            f_P2_L_EDA = P2_L_EDA
            f_P2_R_EDA = P2_R_EDA

    #     if norm_to_baseline:
    #         # Normalize per subject (i.e. standardize) - this global normalisation did not seem to be better than unit normalisation
    #         f_P1_L_EDA = normalize_to_baseline(f_P1_L_EDA, P1_L_baseline)
    #         f_P1_R_EDA = normalize_to_baseline(f_P1_R_EDA, P1_R_baseline)
    #         f_P2_L_EDA = normalize_to_baseline(f_P2_L_EDA, P2_L_baseline)
    #         f_P2_R_EDA = normalize_to_baseline(f_P2_R_EDA, P2_R_baseline)

        # Iterate over annotated events for this game
        # Event files have structure: 
        # Event timestamp 
        #     - system time of the event determined from video recording.
        # Event type 
        #     - D=discarding a card, S=being suspicious (calling "Cheat"), EOG=end of game
        # P01 
        #     - the suit (S=spades/C=clubs/H=hearts/D=diamonds) played by player 1 if event type is D. (if it is not P01's turn then '-'). If event type is S, then 'CH' means that P01 called "Cheat". 
        #     - analogously for P02
        # True suit
        #     - true suit (S=spades/C=clubs/H=hearts/D=diamonds) claimed by a player
        #       (constant for one pile, until "Cheat" was called)

        valid_epoch = False
        events_data = np.loadtxt(game_events_file_name, delimiter=",", skiprows=1, dtype=str)
        for i, event in enumerate(events_data):

            event_type = event[1]

            # After end of game skip the following epoch
            if event_type == 'EOG':
                valid_epoch = False

            else:
                # Check and correct (if needed) timestamp format
                if len(event[0]) == 8 and event[0][2] == ':' and event[0][5] == ':':
                    events_data[i][0] = event[0] + '.000' # system time as string
                elif len(event[0]) == 12 and event[0][2] == ':' and event[0][5] == ':' and event[0][8] == '.':
                    pass
                else:
                    print "Timestamp error ", event

                event_timestamp = events_data[i][0]
                event_P1_label = event[2]
                event_P2_label = event[3]        
                true_suit = event[4]
                if true_suit == '':
                    print "True suit error ", event

                if event_type == 'D':

                    if valid_epoch:

                        if event_P1_label != '': # P1 discarded a card

                            feature_vector = [P1_SID]
                            epoch_start = events_data[i-1][0]
                            epoch_end = event_timestamp
                            epoch_start, epoch_end = delay_epoch(epoch_start, epoch_end, epoch_delay, P1_SID)

                            data_L = [EDA for t,EDA in zip(P1_L_times,f_P1_L_EDA) 
                                      if (epoch_start <= t) and (t <= epoch_end) ]
                            data_R = [EDA for t,EDA in zip(P1_R_times,f_P1_R_EDA) 
                                      if (epoch_start <= t) and (t <= epoch_end) ]

                            # Normalize
                            if znormalize:
                                data_L = znorm(data_L)
                                data_R = znorm(data_R)
                            elif unitnormalize:
                                data_L = unitnorm(data_L)
                                data_R = unitnorm(data_R)

                            # Extract features for P1
                            retL, fL = extract_features(data_L)
    #                         if not retL:
    #                             print "... skipping left-hand  DECEPT features", P1_SID
                            retR, fR = extract_features(data_R)
    #                         if not retR:
    #                             print "... skipping right-hand DECEPT features", P1_SID
                            keepSample = retL and retR
                            feature_vector.extend(fL)
                            feature_vector.extend(fR)

                            if event_P1_label == true_suit:
                                # P1 said truth
                                feature_vector.append(deception_classes[1])
                                feature_vector.append(1)
                            else:
                                # P1 lied
                                feature_vector.append(deception_classes[0])
                                feature_vector.append(0)

                            # Skip subject 05 for deception detection
                            if P1_SID != '05' and keepSample:
                                features_deception.append(feature_vector)

                            ############################# Record trust
                            feature_vector = [P1_SID]
                            epoch_start = events_data[i-1][0]
                            epoch_end = event_timestamp
                            epoch_start, epoch_end = delay_trust_epoch(epoch_start, epoch_end, epoch_delay, P1_SID)

                            data_L = [EDA for t,EDA in zip(P1_L_times,f_P1_L_EDA) 
                                if (epoch_start <= t) and (t <= epoch_end) ]
                            data_R = [EDA for t,EDA in zip(P1_R_times,f_P1_R_EDA) 
                                if (epoch_start <= t) and (t <= epoch_end) ]

                            # Normalize
                            if znormalize:
                                data_L = znorm(data_L)
                                data_R = znorm(data_R)
                            elif unitnormalize:
                                data_L = unitnorm(data_L)
                                data_R = unitnorm(data_R)

                            # Extract features
                            retL, fL = extract_features(data_L)
    #                         if not retL:
    #                             print "... skipping left-hand  TRUST features", P1_SID
                            retR, fR = extract_features(data_R)
    #                         if not retR:
    #                             print "... skipping right-hand TRUST features", P1_SID

                            if retL and retR: # keepSample ?
                                feature_vector.extend(fL)
                                feature_vector.extend(fR)
                                feature_vector.append(suspicion_classes[1])
                                feature_vector.append(1)
                                features_suspicion.append(feature_vector)

                        elif event_P2_label != '': # P2 discarded a card

                            feature_vector = [P2_SID]
                            epoch_start = events_data[i-1][0]
                            epoch_end = event_timestamp
                            epoch_start, epoch_end = delay_epoch(epoch_start, epoch_end, epoch_delay, P2_SID)

                            data_L = [EDA for t,EDA in zip(P2_L_times,f_P2_L_EDA) 
                                      if (epoch_start <= t) and (t <= epoch_end) ]
                            data_R = [EDA for t,EDA in zip(P2_R_times,f_P2_R_EDA) 
                                      if (epoch_start <= t) and (t <= epoch_end) ]

                            # Normalize
                            if znormalize:
                                data_L = znorm(data_L)
                                data_R = znorm(data_R)
                            elif unitnormalize:
                                data_L = unitnorm(data_L)
                                data_R = unitnorm(data_R)

                            # Extract features for P2
                            retL, fL = extract_features(data_L)
    #                         if not retL:
    #                             print "... skipping left-hand  DECEPT features", P2_SID
                            retR, fR = extract_features(data_R)
    #                         if not retR:
    #                             print "... skipping right-hand DECEPT features", P2_SID
                            keepSample = retL and retR
                            feature_vector.extend(fL)
                            feature_vector.extend(fR)

                            if event_P2_label == true_suit:
                                # P2 said truth
                                feature_vector.append(deception_classes[1])
                                feature_vector.append(1)
                            else:
                                # P2 lied     
                                feature_vector.append(deception_classes[0])
                                feature_vector.append(0)

                            if keepSample:
                                features_deception.append(feature_vector)

                            ############################# Record trust
                            feature_vector = [P2_SID]
                            epoch_start = events_data[i-1][0]
                            epoch_end = event_timestamp
                            epoch_start, epoch_end = delay_trust_epoch(epoch_start, epoch_end, epoch_delay, P2_SID)

                            data_L = [EDA for t,EDA in zip(P2_L_times,f_P2_L_EDA) 
                                      if (epoch_start <= t) and (t <= epoch_end) ]
                            data_R = [EDA for t,EDA in zip(P2_R_times,f_P2_R_EDA) 
                                      if (epoch_start <= t) and (t <= epoch_end) ]

                            # Normalize
                            if znormalize:
                                data_L = znorm(data_L)
                                data_R = znorm(data_R)
                            elif unitnormalize:
                                data_L = unitnorm(data_L)
                                data_R = unitnorm(data_R)

                            # Extract features
                            retL, fL = extract_features(data_L)
    #                         if not retL:
    #                             print "... skipping left-hand  TRUST features", P2_SID
                            retR, fR = extract_features(data_R)
    #                         if not retR:
    #                             print "... skipping right-hand TRUST features", P2_SID

                            if retL and retR: # keepSample ?
                                feature_vector.extend(fL)
                                feature_vector.extend(fR)
                                feature_vector.append(suspicion_classes[1])
                                feature_vector.append(1)
                                features_suspicion.append(feature_vector)
                        else:
                            print "Event D ", event, " not labeled"

                    valid_epoch = True

                elif event_type == 'S':
                    numS += 1
                    valid_epoch = False
                    if event_P1_label == 'CH': # P1 said cheat

                        feature_vector = [P1_SID]
                        epoch_start = events_data[i-1][0]
                        epoch_end = event_timestamp
                        epoch_start, epoch_end = delay_susp_epoch(epoch_start, epoch_end, epoch_delay, P1_SID)

                        data_L = [EDA for t,EDA in zip(P1_L_times,f_P1_L_EDA) 
                                  if (epoch_start <= t) and (t <= epoch_end) ]
                        data_R = [EDA for t,EDA in zip(P1_R_times,f_P1_R_EDA) 
                                  if (epoch_start <= t) and (t <= epoch_end) ]

                        # Normalize
                        if znormalize:
                            data_L = znorm(data_L)
                            data_R = znorm(data_R)
                        elif unitnormalize:
                            data_L = unitnorm(data_L)
                            data_R = unitnorm(data_R)

                        # Extract features
                        retL, fL = extract_features(data_L)
    #                     if not retL:
    #                         print "... skipping left-hand  SUSP features", P1_SID
                        retR, fR = extract_features(data_R)
    #                     if not retR:
    #                         print "... skipping right-hand SUSP features", P1_SID

                        if retL and retR: # keepSample ?
                            feature_vector.extend(fL)
                            feature_vector.extend(fR)
                            feature_vector.append(suspicion_classes[0])
                            feature_vector.append(0)
                            features_suspicion.append(feature_vector)

                    elif event_P2_label == 'CH': # P2 said cheat

                        feature_vector = [P2_SID]
                        epoch_start = events_data[i-1][0]
                        epoch_end = event_timestamp
                        epoch_start, epoch_end = delay_susp_epoch(epoch_start, epoch_end, epoch_delay, P2_SID)

                        data_L = [EDA for t,EDA in zip(P2_L_times,f_P2_L_EDA) 
                                  if (epoch_start <= t) and (t <= epoch_end) ]
                        data_R = [EDA for t,EDA in zip(P2_R_times,f_P2_R_EDA) 
                                  if (epoch_start <= t) and (t <= epoch_end) ]

                        # Normalize
                        if znormalize:
                            data_L = znorm(data_L)
                            data_R = znorm(data_R)
                        elif unitnormalize:
                            data_L = unitnorm(data_L)
                            data_R = unitnorm(data_R)

                        # Extract features
                        retL, fL = extract_features(data_L)
    #                     if not retL:
    #                         print "... skipping left-hand  SUSP features", P2_SID
                        retR, fR = extract_features(data_R)
    #                     if not retR:
    #                         print "... skipping right-hand SUSP features", P2_SID

                        if retL and retR: # keepSample ?
                            feature_vector.extend(fL)
                            feature_vector.extend(fR)
                            feature_vector.append(suspicion_classes[0])
                            feature_vector.append(0)
                            features_suspicion.append(feature_vector)
                    else:
                        print "Event S ", event, " not labeled"

                else:
                    print "Event type error in: ", game_events_file_name, "; ", event

    if feature_type == 'deception':
        np.savetxt('./../Experiment/ExtractedFeatures/features_deception_u_' + file_ID + '.csv', 
                   features_deception, delimiter=',', fmt='%s')
#         return np.array(features_deception)
    else:
        np.savetxt('./../Experiment/ExtractedFeatures/features_suspicion_u_' + file_ID + '.csv', 
                   features_suspicion, delimiter=',', fmt='%s')
#         return np.array(features_suspicion)

# Save extracted features: Subject ID | Left-hand features | Right-hand features | Label
# np.savetxt('./../Experiment/ExtractedFeatures/features_deception.csv', features_deception, delimiter=',', fmt='%s')
# np.savetxt('./../Experiment/ExtractedFeatures/features_suspicion.csv', features_suspicion, delimiter=',', fmt='%s')
# num_truths = len([1 for row in features_deception if row[-1] == 1])
# num_decept = len(features_deception) - num_truths
# num_trusts = len([1 for row in features_suspicion if row[-1] == 1])
# num_susp = len(features_suspicion) - num_trusts
# print "Number of examples for deception detection: \t", len(features_deception), "\n\t shape ", np.shape(features_deception)
# print "\t #truths", num_truths, "\t #deceptions", num_decept
# print "Number of examples for suspicion detection: \t", len(features_suspicion), "\n\t shape ", np.shape(features_suspicion)
# print "\t #trusts", num_trusts, "\t #suspicions", num_susp

In [3]:
#######################################################################
# Generate DECEPTION feature files with filenames of format: 
# features_{feature_type}_u_{epoch_delay}_{max_epoch_len}.csv
# u means unit normalization
# epoch_delay and max_epoch_len are indices of arrays:
# eds = np.linspace(1.,4.,7) # Epoch delay ----- delta parameter
# mels = np.linspace(0.5,4.5,9) # Max deception epoch length ----- tau parameter
#######################################################################
import itertools
import time

# Skip subject 05 for deception detection only, not for suspicion detection
eds = np.linspace(1.,4.,7) # Epoch delay
mels = np.linspace(0.5,4.5,9) # Max epoch length
feature_type = 'deception'

# Over all settings of parameters
# for ed, mel, in list(itertools.product(eds, mels)):
for i, ed in enumerate(eds):
    for j, mel in enumerate(mels):
        t = time.time()
        extract_features(feature_type, ed, mel, None, str(i)+'_'+str(j))
        print ed, "\t", mel, ": \t", time.time()-t
        # cca 1 min

/home/janciovec/.local/lib/python2.7/site-packages/ipykernel_launcher.py:69: RuntimeWarning: invalid value encountered in divide


1.0 	0.5 : 	65.2227828503
1.0 	1.0 : 	69.6087179184
1.0 	1.5 : 	72.063117981
1.0 	2.0 : 	71.4052948952
1.0 	2.5 : 	70.8195228577
1.0 	3.0 : 	72.379253149
1.0 	3.5 : 	67.3871240616
1.0 	4.0 : 	68.0320029259
1.0 	4.5 : 	66.4927821159
1.5 	0.5 : 	66.4144070148
1.5 	1.0 : 	66.7719800472
1.5 	1.5 : 	66.0265967846
1.5 	2.0 : 	65.0863518715
1.5 	2.5 : 	65.0311188698
1.5 	3.0 : 	64.6708040237
1.5 	3.5 : 	63.8825669289
1.5 	4.0 : 	66.0186941624
1.5 	4.5 : 	64.6015100479
2.0 	0.5 : 	63.6988139153
2.0 	1.0 : 	64.7224879265
2.0 	1.5 : 	63.7610528469
2.0 	2.0 : 	65.6811199188
2.0 	2.5 : 	65.9122240543
2.0 	3.0 : 	63.8375411034
2.0 	3.5 : 	63.4607388973
2.0 	4.0 : 	65.1350970268
2.0 	4.5 : 	65.1130948067
2.5 	0.5 : 	64.0043811798
2.5 	1.0 : 	70.2871229649
2.5 	1.5 : 	70.4276850224
2.5 	2.0 : 	72.0877010822
2.5 	2.5 : 	72.1555588245
2.5 	3.0 : 	70.364109993
2.5 	3.5 : 	66.8904559612
2.5 	4.0 : 	66.477380991
2.5 	4.5 : 	66.3604719639
3.0 	0.5 : 	66.6896760464
3.0 	1.0 : 	66.7817511559
3.0 	1.5 : 	66.5

In [4]:
######################################################################
# Generate SUSPICION feature files with filenames of format: 
# features_{feature_type}_u_{epoch_delay}_{max_epoch_len}.csv
# u means unit normalization
# epoch_delay and max_epoch_len are indices of arrays:
# eds = np.linspace(1.,4.,7) # Epoch delay ----- delta parameter
# msels = np.linspace(0.5,4.5,9) # maximum suspicion epoch length - tau parameter
#######################################################################
import itertools
import time

# FOR OPTIMIZING msel

# Skip subject 05 for deception detection only, not for suspicion detection
eds = np.linspace(1.,4.,7) # Epoch delay
msels = np.linspace(0.5,4.5,9) # mean suspicion length
feature_type = 'suspicion'
mel=1. # arbitrary for suspicion features extraction
    
# Over all settings of parameters
for i, ed in enumerate(eds):
    for j, msel in enumerate(msels):
        t = time.time()
        extract_features(feature_type, ed, mel, msel, str(i)+'_'+str(j))
        print ed, "\t", msel, ": \t", time.time()-t
        # cca 1 min
   

/home/janciovec/.local/lib/python2.7/site-packages/ipykernel_launcher.py:75: RuntimeWarning: invalid value encountered in divide


1.0 	0.5 : 	206.130706072
1.0 	1.0 : 	205.813843966
1.0 	1.5 : 	74.3898139
1.0 	2.0 : 	73.1668508053
1.0 	2.5 : 	70.3487589359
1.0 	3.0 : 	71.9408020973
1.0 	3.5 : 	72.2675490379
1.0 	4.0 : 	69.6804871559
1.0 	4.5 : 	70.8273730278
1.5 	0.5 : 	70.1733720303
1.5 	1.0 : 	68.8723680973
1.5 	1.5 : 	68.5874090195
1.5 	2.0 : 	69.3945481777
1.5 	2.5 : 	68.7767131329
1.5 	3.0 : 	70.7609689236
1.5 	3.5 : 	70.6522169113
1.5 	4.0 : 	69.4481389523
1.5 	4.5 : 	69.8093512058
2.0 	0.5 : 	67.0920939445
2.0 	1.0 : 	69.3489770889
2.0 	1.5 : 	67.8809530735
2.0 	2.0 : 	69.9358229637
2.0 	2.5 : 	71.4835641384
2.0 	3.0 : 	77.4224867821
2.0 	3.5 : 	80.3053629398
2.0 	4.0 : 	72.8475837708
2.0 	4.5 : 	73.1227450371
2.5 	0.5 : 	78.8753418922
2.5 	1.0 : 	85.073184967
2.5 	1.5 : 	186.41759491
2.5 	2.0 : 	218.287860155
2.5 	2.5 : 	231.606835127
2.5 	3.0 : 	218.634891033
2.5 	3.5 : 	205.687391996
2.5 	4.0 : 	207.316909075
2.5 	4.5 : 	180.580295801
3.0 	0.5 : 	76.1366438866
3.0 	1.0 : 	77.6508979797
3.0 	1.5 : 	74.69